## Representation Testing

In [234]:
import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import re

simulator = AerSimulator()

Representation

In [235]:
# Parameters
qubits = 3
max_circuit_depth = 5

# Initialize circuit
test_circuit = QuantumCircuit(qubits)

# Gate map
chromosome_qiskit_gate_map = {
    "w": test_circuit.barrier,  # Use barrier for blank "wires"
    "-": None,  # Used for control qubits
    "x": test_circuit.x,
    "y": test_circuit.y,
    "z": test_circuit.z,
    "h": test_circuit.h,
    "s": test_circuit.s,
    "sdg": test_circuit.sdg,
    "t": test_circuit.t,
    "cx": test_circuit.cx,
    "cy": test_circuit.cy,
    "cz": test_circuit.cz,
    "ch": test_circuit.ch,
    "cs": test_circuit.cs,
    "swap": test_circuit.swap,
}

# Example chromosome
test_circuit_chromosome = [
    ['h', 'w', 'w'],
    ['cs(1,0)', '-', 'w'],
    ['cz(2,0)', 'w', '-'],
    ['sdg', 'w', 'w'],
    ['t', 'w', 'w'],
    ['cz(2,0)', 'w', '-'],
    ['s', 'w', 'w'],
    ['swap(2,0)', 'w', '-'],
    ['w', 'h', 'w'],
    ['w', 'cs(2,1)', '-'],
    ['w', 'w', 'h']
]

# Helper to apply gates
for block in test_circuit_chromosome:
    for qubit in range(len(block)):
        gate_spec = block[qubit]
        if gate_spec == "-":
            continue
        elif "(" in gate_spec:
            gate, args = re.match(r"(\w+)\((.+)\)", gate_spec).groups()
            args = list(map(int, args.split(",")))
            chromosome_qiskit_gate_map[gate](*args)
        else:
            chromosome_qiskit_gate_map[gate_spec](qubit)

# Add measurements
test_circuit.measure_all()

# Output
print(test_circuit)
print("Circuit depth:", test_circuit.depth())


        ┌───┐┌───┐   ┌─────┐┌───┐      ┌───┐       ░   ░    ░        ░ ┌─┐   »
   q_0: ┤ H ├┤ S ├─■─┤ Sdg ├┤ T ├────■─┤ S ├────X──░───░────░────────░─┤M├───»
        └─░─┘└─┬─┘ │ └──░──┘└─░─┘ ░  │ └─░─┘ ░  │  ░ ┌───┐┌───┐  ░   ░ └╥┘┌─┐»
   q_1: ──░────■───┼────░─────░───░──┼───░───░──┼──░─┤ H ├┤ S ├──░───░──╫─┤M├»
          ░    ░   │    ░     ░   ░  │   ░   ░  │  ░ └───┘└─┬─┘┌───┐ ░  ║ └╥┘»
   q_2: ──░────░───■────░─────░──────■───░──────X──░────────■──┤ H ├─░──╫──╫─»
          ░    ░        ░     ░          ░         ░           └───┘ ░  ║  ║ »
meas: 3/════════════════════════════════════════════════════════════════╩══╩═»
                                                                        0  1 »
«           
«   q_0: ───
«           
«   q_1: ───
«        ┌─┐
«   q_2: ┤M├
«        └╥┘
«meas: 3/═╩═
«         2 
Circuit depth: 11


Running Circuit

In [ ]:
# Transpile for simulator
simulator = AerSimulator()

# Run and get counts
result = simulator.run(test_circuit, shots=10000, memory=True).result()
counts = result.get_counts(test_circuit)
print(test_circuit)
print(test_circuit.depth())
plot_histogram(counts, title='Bell-State counts')

AerError: 'unknown instruction: cs'